## Database Connection

Connecting to the database:

In [ ]:
from data import DataManager

imdb_data = DataManager(
    dbname='imdb',
    user='samaher',
    password="CodingIsFun++",
    host='localhost',
    port='5432'
)

Creating 5 dataframes for the tables I extracted from the database: 
- movie
- genre
- prod
- rating
- crew

In [ ]:
# Get data from the movie table
movie_data = imdb_data.get_movie()

# Get data from the genre table
genre_data = imdb_data.get_genre()

# Get data from the produced table
prod_data = imdb_data.get_prod()

# Get data from the rating table
rating_data = imdb_data.get_rating()

# Get data from the crew table
crew_data = imdb_data.get_crew()


## Libraries

All libraries used will be imported here:

In [ ]:
import pandas as pd
import numpy as np
import random

### Quiz info 

- questions: list of questions from the table movie
- correct answer: the cell in the corresponding row of the column 'year' in the movie table
- options: 3 random options from random cells in the column 'year' in the movie table BUT different from the correct answer
- level: choose the level of difficulty

#### Score

The score is calculated this way: 
wrong answer: 0 points
correct answer:
- 1 if EASY
- 2 if MEDIUM
- 3 if HARD

#### Difficulty Level

- easy: if the movie was released starting from 2010 and has more than 500000 votes
- medium: if the movie was released between 1990 and 2010 and has between 100000 abd 500000
- hard: if the movie was released before 1990 and has less than 100000 votes

In [ ]:
# function for the difficulty level

def determine_difficulty_level(year, votes):
    if (year >= 2010 and votes >= 500000) or (year < 2010 and votes >= 1000000):
        return 'easy'
    elif 1990 <= year < 2010 and 100000 <= votes < 500000:
        return 'medium'
    elif year < 1990 and votes < 100000:
        return 'hard'
    else:
        return 'unknown'


In [ ]:
# function calculating score

def calculate_score(difficulty_level, is_correct):
    if is_correct:
        if difficulty_level == 'easy':
            return 1
        elif difficulty_level == 'medium':
            return 2
        elif difficulty_level == 'hard':
            return 3
    else:
        return 0


In [ ]:

# function generating movie questions
def generate_movie_question(row, desired_difficulty):
    official_title = row['official_title']
    movie_correct_answer = row['year']

    # Determining difficulty level based on release year and votes
    year = int(row['year'])
    votes = int(row['votes'])
    difficulty_level = determine_difficulty_level(year, votes)

    # Check if the difficulty level matches the desired difficulty
    if difficulty_level == desired_difficulty:
        # Getting all unique years excluding the correct answer
        all_years = list(set(movie_data['year'].unique()))
        all_years.remove(movie_correct_answer)  # Removing the correct answer

        # Selecting 3 more random incorrect options
        movie_options = [movie_correct_answer] + random.sample(all_years, 3)

        # Shuffling the options to randomize their order
        random.shuffle(movie_options)

        # Mapping options to letters (A, B, C, D)
        options_mapping = {chr(ord('A') + i): option for i, option in enumerate(movie_options)}

        # Constructing the question dictionary
        question_dict = {
            'question': f'When was the movie {official_title} released?',
            'options': options_mapping,
            'correct_answer': chr(ord('A') + movie_options.index(movie_correct_answer)),
            'difficulty_level': difficulty_level
        }

        return question_dict
    else:
        return None  # Return None for questions with undesired difficulty

In [ ]:
# function for the quiz
def quiz_game():
    difficulty_levels = ['easy', 'medium', 'hard']

    # Getting user input for difficulty level
    user_difficulty = input("Choosing a difficulty level (easy, medium, hard): ").lower()

    # Validating user input
    while user_difficulty not in difficulty_levels:
        print("Invalid difficulty level. Please choose from: easy, medium, hard")
        user_difficulty = input("Choosing a difficulty level (easy, medium, hard): ").lower()

    # Generating a random row index
    while True:
        row_index = random.randint(0, len(movie_data) - 1)
        row = movie_data.iloc[row_index]

        # Generating the question with the desired difficulty level
        question_info = generate_movie_question(row, user_difficulty)

        if question_info is not None:  # Check if a valid question is generated
            break

    # Printing the question information
    print(question_info['question'])

    # Printing options with letters (A, B, C, D)
    for letter, option in question_info['options'].items():
        print(f"{letter}. {option}")

    # Getting user's choice
    user_choice = input("Entering your choice (A, B, C, D): ").upper()

    # Checking if the user's choice is correct
    is_correct = user_choice == question_info['correct_answer']

    # Calculating and displaying the score
    score = calculate_score(question_info['difficulty_level'], is_correct)

    # Providing feedback on the answer
    if is_correct:
        print("Correct!")
    else:
        print(f"Wrong! The correct answer is: {question_info['correct_answer']}")

    print(f"Your score: {score}")


## 10 Question Quiz - V0
Instead of 1 question now 10 questions - This is V0 of the quiz

In [ ]:
def quiz_game():
    difficulty_levels = ['easy', 'medium', 'hard']
    total_score = 0

    # Getting user input for difficulty level
    user_difficulty = input("Choosing a difficulty level (easy, medium, hard): ").lower()

    # Validating user input
    while user_difficulty not in difficulty_levels:
        print("Invalid difficulty level. Please choose from: easy, medium, hard")
        user_difficulty = input("Choosing a difficulty level (easy, medium, hard): ").lower()

    # Playing 10 questions
    for i in range(10):
        while True:
            row_index = random.randint(0, len(movie_data) - 1)
            row = movie_data.iloc[row_index]

            # Generating the question with the desired difficulty level
            question_info = generate_movie_question(row, user_difficulty)

            if question_info is not None:  # Check if a valid question is generated
                break

        # Printing the question information
        print(question_info['question'])

        # Printing options with letters (A, B, C, D)
        for letter, option in question_info['options'].items():
            print(f"{letter}. {option}")

        # Getting user's choice
        user_choice = input("Entering your choice (A, B, C, D): ").upper()

        # Checking if the user's choice is correct
        is_correct = user_choice == question_info['correct_answer']

        # Calculating and displaying the score
        score = calculate_score(question_info['difficulty_level'], is_correct)
        total_score += score

        # Providing feedback on the answer
        if is_correct:
            print("Correct!")
        else:
            print(f"Wrong! The correct answer is: {question_info['correct_answer']}")

        print(f"Your score for this question: {score}")
        print("----------------------------")

    print(f"Total score: {total_score}")

## Updated Version - No repeated questions
I noticed that the same question can appear twice in the same quiz. 

In [ ]:
def quiz_game():
    difficulty_levels = ['easy', 'medium', 'hard']
    total_score = 0
    used_questions = []

    # Getting user input for difficulty level
    user_difficulty = input("Choosing a difficulty level (easy, medium, hard): ").lower()

    # Validating user input
    while user_difficulty not in difficulty_levels:
        print("Invalid difficulty level. Please choose from: easy, medium, hard")
        user_difficulty = input("Choosing a difficulty level (easy, medium, hard): ").lower()

    # Playing 10 questions
    for i in range(10):
        while True:
            row_index = random.randint(0, len(movie_data) - 1)
            row = movie_data.iloc[row_index]

            # Generating the question with the desired difficulty level
            question_info = generate_movie_question(row, user_difficulty)

            if question_info is not None and question_info['question'] not in used_questions:
                used_questions.append(question_info['question'])
                break

        # Printing the question information
        print(question_info['question'])

        # Printing options with letters (A, B, C, D)
        for letter, option in question_info['options'].items():
            print(f"{letter}. {option}")

        # Getting user's choice
        user_choice = input("Entering your choice (A, B, C, D): ").upper()

        # Checking if the user's choice is correct
        is_correct = user_choice == question_info['correct_answer']

        # Calculating and displaying the score
        score = calculate_score(question_info['difficulty_level'], is_correct)
        total_score += score

        # Providing feedback on the answer
        if is_correct:
            print("Correct!")
        else:
            print(f"Wrong! The correct answer is: {question_info['correct_answer']}")

        print(f"Your score for this question: {score}")
        print("----------------------------")

    print(f"Total score: {total_score}")


## Limit User Input for the answers - V1

Users should only enter a b c or d, else, they should be notified of the error.

In [ ]:
def quiz_game():
    difficulty_levels = ['easy', 'medium', 'hard']
    total_score = 0
    used_questions = []

    # Getting user input for difficulty level
    user_difficulty = input("Choosing a difficulty level (easy, medium, hard): ").lower()

    # Validating user input
    while user_difficulty not in difficulty_levels:
        print("Invalid difficulty level. Please choose from: easy, medium, hard")
        user_difficulty = input("Choosing a difficulty level (easy, medium, hard): ").lower()

    # Playing 10 questions
    for i in range(10):
        while True:
            row_index = random.randint(0, len(movie_data) - 1)
            row = movie_data.iloc[row_index]

            # Generating the question with the desired difficulty level
            question_info = generate_movie_question(row, user_difficulty)

            if question_info is not None and question_info['question'] not in used_questions:
                used_questions.append(question_info['question'])
                break

        # Printing the question information
        print(question_info['question'])

        # Printing options with letters (A, B, C, D)
        for letter, option in question_info['options'].items():
            print(f"{letter}. {option}")

        # Getting user's choice with validation
        user_choice = None
        while user_choice not in ['A', 'B', 'C', 'D']:
            user_choice = input("Entering your choice (A, B, C, D): ").upper()

            if user_choice not in ['A', 'B', 'C', 'D']:
                print("You typed a wrong letter. Please type again.")

        # Checking if the user's choice is correct
        is_correct = user_choice == question_info['correct_answer']

        # Calculating and displaying the score
        score = calculate_score(question_info['difficulty_level'], is_correct)
        total_score += score

        # Providing feedback on the answer
        if is_correct:
            print("Correct!")
        else:
            print(f"Wrong! The correct answer is: {question_info['correct_answer']}")

        print(f"Your score for this question: {score}")
        print("----------------------------")

    print(f"Total score: {total_score}")


## Adding a 2nd Question Type - V2

So far, I have only question of release dates. I will add prod questions. That's why I need to make the function that generates questions more generic.

In [ ]:
def generate_question(row, desired_difficulty, data_frame, question_type, correct_answer_column):
    official_title = row['official_title']
    correct_answer = row[correct_answer_column]

    # Determining difficulty level 
    year = int(row['year']) 
    votes = int(row['votes'])
    difficulty_level = determine_difficulty_level(year, votes)

    # Check if the difficulty level matches the desired difficulty
    if difficulty_level == desired_difficulty:
        # Getting unique values excluding the correct answer
        all_answers = list(set(data_frame[correct_answer_column].unique()))
        all_answers.remove(correct_answer)  # Removing the correct answer

        # Selecting 3 more random incorrect options
        other_options = [correct_answer] + random.sample(all_answers, 3)

        # Shuffling the options to randomize their order
        random.shuffle(other_options)

        # Mapping options to letters (A, B, C, D)
        options_mapping = {chr(ord('A') + i): option for i, option in enumerate(other_options)}

        # Constructing the question dictionary
        question_dict = {
            'question': f'{question_type} || {official_title} ||',
            'options': options_mapping,
            'correct_answer': chr(ord('A') + other_options.index(correct_answer)),
            'difficulty_level': difficulty_level
        }

        return question_dict


In [ ]:
def quiz_game():
    difficulty_levels = ['easy', 'medium', 'hard']
    total_score = 0
    used_questions = []

    # Getting user input for difficulty level
    user_difficulty = input("Choosing a difficulty level (easy, medium, hard): ").lower()

    # Validating user input
    while user_difficulty not in difficulty_levels:
        print("Invalid difficulty level. Please choose from: easy, medium, hard")
        user_difficulty = input("Choosing a difficulty level (easy, medium, hard): ").lower()

    # List of question generators with parameters for different question types
    question_generators = [
        {'generator': generate_question, 'params': (movie_data, 'When was this movie released? ==> ', 'year')},
        {'generator': generate_question, 'params': (prod_data, 'Where was this movie produced? ==> ', 'country')}
        # Add more question types with respective data and attributes
    ]

    # Playing 10 questions, randomly choosing from different types of questions
    for i in range(3):
        while True:
            # Randomly select a question generator with its respective parameters
            question_info = None
            generator_info = random.choice(question_generators)
            data_frame, question_type, correct_answer_column = generator_info['params']

            row_index = random.randint(0, len(data_frame) - 1)
            row = data_frame.iloc[row_index]

            # Generating the question with the desired difficulty level using the selected generator
            question_info = generator_info['generator'](row, user_difficulty, data_frame, question_type, correct_answer_column)

            if question_info is not None and question_info['question'] not in used_questions:
                used_questions.append(question_info['question'])
                break

        # Printing the question information
        print(question_info['question'])

        # Printing options with letters (A, B, C, D)
        for letter, option in question_info['options'].items():
            print(f"{letter}. {option}")

        # Getting user's choice with validation
        user_choice = None
        while user_choice not in ['A', 'B', 'C', 'D']:
            user_choice = input("Entering your choice (A, B, C, D): ").upper()

            if user_choice not in ['A', 'B', 'C', 'D']:
                print("You typed a wrong letter. Please type again.")

        # Checking if the user's choice is correct
        is_correct = user_choice == question_info['correct_answer']

        # Calculating and displaying the score
        score = calculate_score(question_info['difficulty_level'], is_correct)
        total_score += score

        # Providing feedback on the answer
        if is_correct:
            print("Correct!")
        else:
            print(f"Wrong! The correct answer is: {question_info['correct_answer']}")

        print(f"Your score for this question: {score}")
        print("----------------------------")

    print(f"Total score: {total_score}")


## Question Type 3 

Which movie among these 4 options has the highest score in IMDb?

In [36]:
# Updated function with 3 types

def generate_question(row, desired_difficulty, data_frame, question_type, correct_answer_column):
    if question_type == 'highest_imdb_score':
        max_score_row = data_frame.loc[data_frame['score'].idxmax()]
        correct_answer = max_score_row['official_title']
        
        other_movies = data_frame[data_frame['official_title'] != correct_answer]
        other_options = other_movies['official_title'].sample(n=3).tolist()
        
        options = [correct_answer] + other_options
        np.random.shuffle(options)
        
        options_mapping = {chr(ord('A') + i): option for i, option in enumerate(options)}
        
        question_dict = {
            'question': f'Which movie among these has the highest IMDb score?',
            'options': options_mapping,
            'correct_answer': chr(ord('A') + options.index(correct_answer)),
            'difficulty_level': determine_difficulty_level(row['year'], row['votes'])
        }
        return question_dict

    # For movie and prod types of questions
    official_title = row['official_title']
    correct_answer = row[correct_answer_column]
    year = int(row['year'])
    votes = int(row['votes'])
    difficulty_level = determine_difficulty_level(year, votes)

    if difficulty_level == desired_difficulty:
        all_answers = list(set(data_frame[correct_answer_column].unique()))
        all_answers.remove(correct_answer)

        other_options = [correct_answer] + random.sample(all_answers, 3)
        random.shuffle(other_options)

        options_mapping = {chr(ord('A') + i): option for i, option in enumerate(other_options)}

        question_dict = {
            'question': f'{question_type} || {official_title} ||',
            'options': options_mapping,
            'correct_answer': chr(ord('A') + other_options.index(correct_answer)),
            'difficulty_level': difficulty_level
        }
        return question_dict




In [37]:
def quiz_game():
    difficulty_levels = ['easy', 'medium', 'hard']
    total_score = 0
    used_questions = []

    # Getting user input for difficulty level
    user_difficulty = input("Choosing a difficulty level (easy, medium, hard): ").lower()

    # Validating user input
    while user_difficulty not in difficulty_levels:
        print("Invalid difficulty level. Please choose from: easy, medium, hard")
        user_difficulty = input("Choosing a difficulty level (easy, medium, hard): ").lower()

    # List of question generators with parameters for different question types
    question_generators = [
        {'generator': generate_question, 'params': (movie_data, 'When was this movie released? ==> ', 'year'), 'question_type': 'movie'},
        {'generator': generate_question, 'params': (prod_data, 'Where was this movie produced? ==> ', 'country'), 'question_type': 'prod'},
        {'generator': generate_question, 'params': (rating_data, 'Which movie has the highest IMDb score? ==> ', 'highest_imdb_score'), 'question_type': 'rating'}
    ]

    # Playing 10 questions, randomly choosing from different types of questions
    for i in range(3):
        while True:
            # Randomly select a question generator with its respective parameters
            question_info = None
            generator_info = random.choice(question_generators)
            data_frame, question_type, *correct_answer_column = generator_info['params']
            # The '*' operator gathers additional values as a list, allowing for flexibility in the number of parameters

            row_index = random.randint(0, len(data_frame) - 1)
            row = data_frame.iloc[row_index]

            # Generating the question with the desired difficulty level using the selected generator
            question_info = generator_info['generator'](row, user_difficulty, data_frame, question_type, correct_answer_column)

            if question_info is not None and question_info['question'] not in used_questions:
                used_questions.append(question_info['question'])
                break

        # Printing the question information
        print(question_info['question'])

        # Printing options with letters (A, B, C, D)
        for letter, option in question_info['options'].items():
            print(f"{letter}. {option}")

        # Getting user's choice with validation
        user_choice = None
        while user_choice not in ['A', 'B', 'C', 'D']:
            user_choice = input("Entering your choice (A, B, C, D): ").upper()

            if user_choice not in ['A', 'B', 'C', 'D']:
                print("You typed a wrong letter. Please type again.")

        # Checking if the user's choice is correct
        is_correct = user_choice == question_info['correct_answer']

        # Calculating and displaying the score
        score = calculate_score(question_info['difficulty_level'], is_correct)
        total_score += score

        # Providing feedback on the answer
        if is_correct:
            print("Correct!")
        else:
            print(f"Wrong! The correct answer is: {question_info['correct_answer']}")

        print(f"Your score for this question: {score}")
        print("----------------------------")

    print(f"Total score: {total_score}")


again

In [39]:
# Updated function with 3 types

def generate_question(row, desired_difficulty, data_frame, question_type, correct_answer_column):
    if question_type == 'rating':
        max_score_row = data_frame.loc[data_frame['score'].idxmax()]
        correct_answer = max_score_row['official_title']
        
        other_movies = data_frame[data_frame['official_title'] != correct_answer]
        other_options = other_movies['official_title'].sample(n=3).tolist()
        
        options = [correct_answer] + other_options
        np.random.shuffle(options)
        
        options_mapping = {chr(ord('A') + i): option for i, option in enumerate(options)}
        
        question_dict = {
            'question': f'Which movie among these has the highest IMDb score?',
            'options': options_mapping,
            'correct_answer': chr(ord('A') + options.index(correct_answer)),
            'difficulty_level': determine_difficulty_level(row['year'], row['votes'])
        }
        return question_dict

    # For movie and prod types of questions
    official_title = row['official_title']
    correct_answer = row[correct_answer_column]
    year = int(row['year'])
    votes = int(row['votes'])
    difficulty_level = determine_difficulty_level(year, votes)

    if difficulty_level == desired_difficulty:
        all_answers = list(set(data_frame[correct_answer_column].unique()))
        all_answers.remove(correct_answer)

        other_options = [correct_answer] + random.sample(all_answers, 3)
        random.shuffle(other_options)

        options_mapping = {chr(ord('A') + i): option for i, option in enumerate(other_options)}

        question_dict = {
            'question': f'{question_type} || {official_title} ||',
            'options': options_mapping,
            'correct_answer': chr(ord('A') + other_options.index(correct_answer)),
            'difficulty_level': difficulty_level
        }
        return question_dict

In [40]:

def quiz_game():
    difficulty_levels = ['easy', 'medium', 'hard']
    total_score = 0
    used_questions = []

    # Getting user input for difficulty level
    user_difficulty = input("Choosing a difficulty level (easy, medium, hard): ").lower()

    # Validating user input
    while user_difficulty not in difficulty_levels:
        print("Invalid difficulty level. Please choose from: easy, medium, hard")
        user_difficulty = input("Choosing a difficulty level (easy, medium, hard): ").lower()

    # List of question generators with parameters for different question types
    question_generators = [
        {'generator': generate_question, 'params': (movie_data, 'When was this movie released? ==> ', 'year'), 'question_type': 'movie'},
        {'generator': generate_question, 'params': (prod_data, 'Where was this movie produced? ==> ', 'country'), 'question_type': 'prod'},
        {'generator': generate_question, 'params': (rating_data, 'Which movie has the highest IMDb score? ==> ', 'score'), 'question_type': 'rating'}
        # Add more question types with respective data and attributes
    ]

    # Playing 10 questions, randomly choosing from different types of questions
    for i in range(3):
        while True:
            # Randomly select a question generator with its respective parameters
            question_info = None
            generator_info = random.choice(question_generators)
            data_frame, question_type, *correct_answer_column = generator_info['params']
            # The '*' operator gathers additional values as a list, allowing for flexibility in the number of parameters

            row_index = random.randint(0, len(data_frame) - 1)
            row = data_frame.iloc[row_index]

            # Generating the question with the desired difficulty level using the selected generator
            question_info = generator_info['generator'](row, user_difficulty, data_frame, question_type, correct_answer_column)

            if question_info is not None and question_info['question'] not in used_questions:
                used_questions.append(question_info['question'])
                break

        # Printing the question information
        print(question_info['question'])

        # Printing options with letters (A, B, C, D)
        for letter, option in question_info['options'].items():
            print(f"{letter}. {option}")

        # Getting user's choice with validation
        user_choice = None
        while user_choice not in ['A', 'B', 'C', 'D']:
            user_choice = input("Entering your choice (A, B, C, D): ").upper()

            if user_choice not in ['A', 'B', 'C', 'D']:
                print("You typed a wrong letter. Please type again.")

        # Checking if the user's choice is correct
        is_correct = user_choice == question_info['correct_answer']

        # Calculating and displaying the score
        score = calculate_score(question_info['difficulty_level'], is_correct)
        total_score += score

        # Providing feedback on the answer
        if is_correct:
            print("Correct!")
        else:
            print(f"Wrong! The correct answer is: {question_info['correct_answer']}")

        print(f"Your score for this question: {score}")
        print("----------------------------")

    print(f"Total score: {total_score}")


## Checking the quiz

In [41]:
# Try the quiz 
quiz_game()

AttributeError: 'DataFrame' object has no attribute 'unique'

## Closing the connection:

In [ ]:
imdb_data.close_connection()